In [1]:
import numpy as np
import amstrax
import strax
import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient
%matplotlib inline
%load_ext line_profiler

/Users/petergaemers/Desktop/python/straxen/straxen/plugins/fax.py:142: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "add_current" failed type inference due to: Untyped global name '_template_length': cannot determine Numba type of <class 'numba.ir.UndefinedType'>

File "../../straxen/straxen/plugins/fax.py", line 153:
    def add_current(_photon_timing_start, _reminder, _photon_gain, pulse):
        <source elided>
        #         """
        pulse[_photon_timing_start:_photon_timing_start + _template_length] += \
        ^

  @guvectorize([(int64, int64, float64, float64[:])], '(), (), (), (n)')
/anaconda3/envs/amstrax/lib/python3.6/site-packages/numba-0.44.1-py3.6-macosx-10.7-x86_64.egg/numba/compiler.py:725: NumbaWarning: Function "add_current" was compiled in object mode without forceobj=True.

File "../../straxen/straxen/plugins/fax.py", line 143:
    @guvectorize([(int64, int64, float64, float64[:])], '(), (), (), (n)')


In [2]:
%matplotlib inline

In [3]:
# run_id = '190516_095707'_
run_id = '190610_085915'

In [4]:
st = strax.Context(storage=[strax.DataDirectory('./data'),
                           strax.DataDirectory('./raw',readonly= True),
                           ],
                  register=[amstrax.RecordsFromMongo, amstrax.PulseProcessing, amstrax.Peaks, amstrax.TriggerPeaks,
                            amstrax.PeakBasics,amstrax.PeakClassification,amstrax.PeakPositions,amstrax.NCompeting,
                            amstrax.Events,amstrax.EventBasics, amstrax.EventPositions, amstrax.EventInfo],
                    register_all=amstrax,
#                    config=dict(pax_raw_dir='/data/xenon/xams/run10/raw/data/'),
                  )

In [5]:
st.get_df(run_id, 'event_info')

,event_number,time,endtime,n_peaks,drift_time,s1_index,s1_area,s1_area_fraction_top,s1_range_50p_area,s1_n_competing,...,s2_area_fraction_top,s2_range_50p_area,s2_n_competing,x_s2,y_s2,s2_largest_other,s1_largest_other,alt_s1_interaction_drift_time,xr,yr
0,0,678606132,680608610,22,3888,0,7441.000000,1.0,11.708333,22,...,1.0,587.917603,0,0.0,0.0,0.000000e+00,0.000000,0.0,-4.651312,0.536797
1,1,13792593430,13794607726,16,61394,1,10970.160156,1.0,30.020514,16,...,1.0,459.313354,0,0.0,0.0,0.000000e+00,8078.799805,338.0,-4.082508,-0.192146
2,2,14335075132,14337105042,22,60854,0,9949.679688,1.0,14.857894,21,...,1.0,947.372192,1,0.0,0.0,4.186221e+06,0.000000,0.0,-12.817703,9.491159
3,3,18300678158,18302797956,1,0,-1,0.000000,0.0,0.000000,0,...,1.0,56604.742188,0,0.0,0.0,0.000000e+00,0.000000,0.0,-999.000000,-999.000000
4,4,20957478372,20959503190,3,0,1,12458.360352,1.0,17.250000,3,...,0.0,0.000000,0,0.0,0.0,0.000000e+00,8461.480469,0.0,0.000000,0.000000
5,5,21632426596,21634505298,28,1070,1,7185.879883,1.0,17.663462,28,...,1.0,1456.698730,0,0.0,0.0,0.000000e+00,0.000000,0.0,4.159239,14.792323
6,6,27112567720,27114652502,49,5896,3,23300.960938,1.0,17.815388,48,...,1.0,8837.058594,0,0.0,0.0,0.000000e+00,16455.240234,4970.0,13.277576,7.819604
7,7,27288859708,27290879570,32,10686,0,8886.679688,1.0,13.805554,32,...,1.0,558.177979,0,0.0,0.0,0.000000e+00,0.000000,0.0,-999.000000,-999.000000
8,8,27302110480,27304164422,58,19922,0,14669.400391,1.0,23.546570,48,...,1.0,436.289093,0,0.0,0.0,0.000000e+00,12075.679688,14220.0,-16.392689,-9.519583
9,9,35120991376,35122993524,22,4642,0,13308.759766,1.0,29.075396,22,...,1.0,322.229889,0,0.0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000


In [6]:
# !ls ./data

In [7]:
! rm -r ./data/190610_085915*

In [8]:
# r = st.get_array(run_id, 'records')

In [9]:
from SiPMdata import *

In [10]:
class EventPositions(strax.LoopPlugin):
    depends_on = ('events','event_basics','peaks', 'peak_classification')
    dtype = [
        ('xr', np.float32,
         'Interaction x-position'),
        ('yr', np.float32,
         'Interaction y-position'),
    ]

    def setup(self):
        # z position of the in-plane SiPMs
        z_plane = 10
        # radius of the cyinder for SiPMs at the side
        r_cylinder = 22
        # radius of a SiPM - I assume circular SiPMs with a radius to make the area correspond to a 3x3mm2 square.
        r_sipm = 1.6925
        # build geometry
        geo = GeoParameters(z_plane=z_plane, r_cylinder=r_cylinder, r_sipm=r_sipm)

        sipm = SiPM(type="plane", position=[0, -15, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-13, -7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[13, -7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-4, 0, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[4, 0, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[-13, 7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        sipm = SiPM(type="plane", position=[13, 7.5, z_plane], qeff=0.25)
        geo.add_sipm(sipm)
        
        self.geo = geo

    def compute_loop(self, events, peaks):
        result = dict()
        
        if not len(peaks):
            return result
        
        s2_index = events['s2_index']
        if s2_index == -1 or s2_index > len(peaks[(peaks['type']==2)]) -1 :
            return result
        
        s2_peak = peaks[(peaks['type']==2)][s2_index]
        for i, area in enumerate(s2_peak['area_per_channel'][:7]):
            self.geo.sipms[i].set_number_of_hits(area)

        posrec = Reconstruction(self.geo)
        pos = posrec.reconstruct_position('CHI2')
        for key in ['xr','yr']:
            result[key] = pos[key]
        return result


In [11]:
ep = st.get_array(run_id,'event_positions',
                 register = EventPositions)

Using TensorFlow backend.


<SiPMdata.SiPM object at 0x1315e4cc0>  n =  52744740.0
<SiPMdata.SiPM object at 0x131556be0>  n =  1241616.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  164066190.0
<SiPMdata.SiPM object at 0x131664208>  n =  24510526.0
<SiPMdata.SiPM object at 0x131664748>  n =  68743990.0
<SiPMdata.SiPM object at 0x131664470>  n =  6402859.5
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  342030.88
<SiPMdata.SiPM object at 0x131556be0>  n =  21345.041
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  106257.484
<SiPMdata.SiPM object at 0x131664208>  n =  7696.12
<SiPMdata.SiPM object at 0x131664748>  n =  98986.56
<SiPMdata.SiPM object at 0x131664470>  n =  21557.639
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  362015.28
<SiPMdata.SiPM object at 0x131556be0>  n =  307504.66
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  1647522.4
<SiPMdata

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  0.0
<SiPMdata.SiPM object at 0x131556be0>  n =  8886.68
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  16752.88
<SiPMdata.SiPM object at 0x131664208>  n =  0.0
<SiPMdata.SiPM object at 0x131664748>  n =  9056.76
<SiPMdata.SiPM object at 0x131664470>  n =  0.0
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  9462912.0
<SiPMdata.SiPM object at 0x131556be0>  n =  7424162.5
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  1731797.2
<SiPMdata.SiPM object at 0x131664208>  n =  2750023.5
<SiPMdata.SiPM object at 0x131664748>  n =  1752716.9
<SiPMdata.SiPM object at 0x131664470>  n =  1595010.1
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  -9609.52
<SiPMdata.SiPM object at 0x131556be0>  n =  -0.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  21302.52
<SiPMdata.SiPM object at 0x131664208>  n =

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  1486074.1
<SiPMdata.SiPM object at 0x131556be0>  n =  717907.6
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  330678.06
<SiPMdata.SiPM object at 0x131664208>  n =  432981.16
<SiPMdata.SiPM object at 0x131664748>  n =  445184.38
<SiPMdata.SiPM object at 0x131664470>  n =  298617.94
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  3508070.0
<SiPMdata.SiPM object at 0x131556be0>  n =  1751159.6
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  58315240.0
<SiPMdata.SiPM object at 0x131664208>  n =  2255090.5
<SiPMdata.SiPM object at 0x131664748>  n =  6999602.0
<SiPMdata.SiPM object at 0x131664470>  n =  -21897.805
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  841768.4
<SiPMdata.SiPM object at 0x131556be0>  n =  186875.39
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  2565487.0
<SiPMdata.Si

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  834242.3
<SiPMdata.SiPM object at 0x131556be0>  n =  241088.42
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  3140017.0
<SiPMdata.SiPM object at 0x131664208>  n =  98136.16
<SiPMdata.SiPM object at 0x131664748>  n =  564495.56
<SiPMdata.SiPM object at 0x131664470>  n =  166210.67
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  241513.61
<SiPMdata.SiPM object at 0x131556be0>  n =  42860.16
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  1328707.5
<SiPMdata.SiPM object at 0x131664208>  n =  29508.88
<SiPMdata.SiPM object at 0x131664748>  n =  46006.64
<SiPMdata.SiPM object at 0x131664470>  n =  48897.996
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  808220.2
<SiPMdata.SiPM object at 0x131556be0>  n =  191722.67
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  2665621.5
<SiPMdata.SiPM obj

...

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  466529.44
<SiPMdata.SiPM object at 0x131556be0>  n =  246360.88
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  3256861.8
<SiPMdata.SiPM object at 0x131664208>  n =  358741.25
<SiPMdata.SiPM object at 0x131664748>  n =  1069548.0
<SiPMdata.SiPM object at 0x131664470>  n =  123478.086
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  1160540.9
<SiPMdata.SiPM object at 0x131556be0>  n =  258776.7
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  406491.22
<SiPMdata.SiPM object at 0x131664208>  n =  247253.8
<SiPMdata.SiPM object at 0x131664748>  n =  526142.5
<SiPMdata.SiPM object at 0x131664470>  n =  132832.48
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  1081751.2
<SiPMdata.SiPM object at 0x131556be0>  n =  1013124.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  3841214.2
<SiPMdata.SiPM

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  143292.4
<SiPMdata.SiPM object at 0x131556be0>  n =  103578.72
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  246063.23
<SiPMdata.SiPM object at 0x131664208>  n =  351725.44
<SiPMdata.SiPM object at 0x131664748>  n =  409297.53
<SiPMdata.SiPM object at 0x131664470>  n =  344326.97
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  1481864.5
<SiPMdata.SiPM object at 0x131556be0>  n =  145545.97
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  1837416.8
<SiPMdata.SiPM object at 0x131664208>  n =  278676.06
<SiPMdata.SiPM object at 0x131664748>  n =  427878.75
<SiPMdata.SiPM object at 0x131664470>  n =  178626.5
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  3425379.2
<SiPMdata.SiPM object at 0x131556be0>  n =  831988.94
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  4025752.2
<SiPMdata.SiPM

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  0.0
<SiPMdata.SiPM object at 0x131556be0>  n =  18921.4
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  14286.72
<SiPMdata.SiPM object at 0x131664208>  n =  0.0
<SiPMdata.SiPM object at 0x131664748>  n =  0.0
<SiPMdata.SiPM object at 0x131664470>  n =  0.0
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  2689432.8
<SiPMdata.SiPM object at 0x131556be0>  n =  728877.9
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  635418.94
<SiPMdata.SiPM object at 0x131664208>  n =  664545.06
<SiPMdata.SiPM object at 0x131664748>  n =  693926.4
<SiPMdata.SiPM object at 0x131664470>  n =  133470.27
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  616752.56
<SiPMdata.SiPM object at 0x131556be0>  n =  368308.25
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  4873131.5
<SiPMdata.SiPM object at 0x131664208>  n 

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  70158.0
<SiPMdata.SiPM object at 0x131556be0>  n =  24619.082
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  481411.4
<SiPMdata.SiPM object at 0x131664208>  n =  67861.92
<SiPMdata.SiPM object at 0x131664748>  n =  115484.31
<SiPMdata.SiPM object at 0x131664470>  n =  7526.04
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  96831940.0
<SiPMdata.SiPM object at 0x131556be0>  n =  3451051.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  320131970.0
<SiPMdata.SiPM object at 0x131664208>  n =  42913260.0
<SiPMdata.SiPM object at 0x131664748>  n =  78497590.0
<SiPMdata.SiPM object at 0x131664470>  n =  7114359.0
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  0.0
<SiPMdata.SiPM object at 0x131556be0>  n =  0.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  17433.2
<SiPMdata.SiPM object at 0x

<SiPMdata.SiPM object at 0x1315e4cc0>  n =  1847068.8
<SiPMdata.SiPM object at 0x131556be0>  n =  2096916.2
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  367160.2
<SiPMdata.SiPM object at 0x131664208>  n =  560923.8
<SiPMdata.SiPM object at 0x131664748>  n =  381276.84
<SiPMdata.SiPM object at 0x131664470>  n =  305421.16
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  0.0
<SiPMdata.SiPM object at 0x131556be0>  n =  0.0
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  0.0
<SiPMdata.SiPM object at 0x131664208>  n =  0.0
<SiPMdata.SiPM object at 0x131664748>  n =  0.0
<SiPMdata.SiPM object at 0x131664470>  n =  0.0
m_status = False
<SiPMdata.SiPM object at 0x1315e4cc0>  n =  347813.56
<SiPMdata.SiPM object at 0x131556be0>  n =  102685.79
<SiPMdata.SiPM object at 0x131679ba8>  n =  0.0
<SiPMdata.SiPM object at 0x1316647f0>  n =  1483692.9
<SiPMdata.SiPM object at 0x131664208>  n =  521295

In [12]:
ep

array([(  -4.651312 ,  5.3679740e-01), (  -4.0825076, -1.9214562e-01),
       ( -12.817703 ,  9.4911594e+00), ...,
       (-999.       , -9.9900000e+02), ( -16.890442 ,  1.1142687e+01),
       (   0.       ,  0.0000000e+00)],
      dtype=[(('Interaction x-position', 'xr'), '<f4'), (('Interaction y-position', 'yr'), '<f4')])